In [1]:
from arcgis.gis import GIS,Group
from arcgis.features import FeatureLayerCollection
import json

In [2]:
gis = GIS("pro")

In [3]:
CONFIGURACION_KEYWORD = "CONFIGURACION"
PARCEL_SERVICE_NAME ="PARCEL_SERVICE_NAME"
EMAIL_DATA_NOTIFICA_ASIGNACION = "EMAIL_DATA_NOTIFICA_ASIGNACION"
jefe = "Alfredo Alarcon-jalarcon_project_esri_co"
reconocedor = "William Rene Moreno-wmoreno_project_esri_co"
numero_actualizacion = "Prueba notificación"

In [4]:
def ObtenerConfiguracion(gis,where_codigo="1=1"):
    """Obtener parametros configurados en la tabla de configuracion del sistema
    Parametros:    
    - {where_codigo}: codigos para hacer la consulta solamente por esos codigos (Ej: where_codigo='CODIGO1','CODIGO2','CODIGO3')
    """ 
    actualizacion_fl = None
    try:        
        #ACA SE USA EL NOMBRE DE LA CREDENCIAL ALMACENADA EN EL OS
        if gis is not None:
            print("Obteniendo parámetros de configuracion....")
            listado = {}
            #if actualizacion_fl is None:
            actualizacion_fl = gis.content.search('Actualizacion_Catastro_2.0','Feature Layer')
            if len(actualizacion_fl)>0:
                config_item = actualizacion_fl[0]
                for table in config_item.tables:
                    if table.properties.name.upper()==CONFIGURACION_KEYWORD:                        
                        query_result = table.query(where=where_codigo, out_fields='*')
                        for feature in query_result.features:    
                            opcion = feature.attributes["opcion"]
                            valor = feature.attributes["valor"]    
                            listado[str(opcion)]=valor
            #print(listado)     
        else:
            print("Gis es nulo")
        return listado
    except Exception as ex:            
        print("Error obteniendo los parametros : {}".format(str(ex)))
        return None

In [5]:
dictParams = ObtenerConfiguracion(gis)
#dictParams["EMAIL_DATA_NOTIFICA_ASIGNACION"]

Obteniendo parámetros de configuracion....


In [6]:
def enviar_correo(segis, asunto, mensaje, email_destino=None, type_text='plain'):
    """Metodo para enviar correos
    Parametros:
    - gis: objeto gis
    - asunto: asunto del mensaje
    - mensaje: cuerpo del correo
    - {email_destino}: email destino
    """
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    try:
        print("Enviando correo electronico a {}...".format(email_destino))
        CON_INFO_EMAIL_DATA = "CON_INFO_EMAIL_DATA"
        dictParams = ObtenerConfiguracion(gis, "opcion='{}'".format(CON_INFO_EMAIL_DATA))
        email_data = json.loads(dictParams[CON_INFO_EMAIL_DATA])
        msg = MIMEMultipart()
        password = email_data["password"]
        msg['From'] = email_data["mailfrom"]
        msg['To'] = email_data["mailsuperusuario"] if email_destino is None else email_destino
        msg['Subject'] = asunto
        msg.attach(MIMEText(mensaje, type_text, 'utf-8'))
        server = smtplib.SMTP(email_data["server"])
        server.starttls()
        server.login(msg['From'], password)
        server.sendmail(msg['From'], [msg['To']] + email_data["cco"], msg.as_string())
        server.quit()
        print("Mensaje enviado satisfactoriamente")
    except Exception as e:
        if 'server' in locals():
            server.quit()
        print("Exception enviar_correo: {}".format(str(e)))

In [7]:
def notificarAsignacion(numero_actualizacion, listado_numeros_prediales, asignado_a, asignado_por):
    strJson = dictParams[EMAIL_DATA_NOTIFICA_ASIGNACION]
    asunto_mensaje = json.loads(strJson)
    usuario = gis.users.get(asignado_a.split("-")[1])
    print("usuario: {}".format(usuario))
    codigos_predios = str(listado_numeros_prediales).replace("[","").replace("]","")
    asunto = asunto_mensaje["asunto"].format(NRO_ASIGNACION=numero_actualizacion)
    mensaje = asunto_mensaje["mensaje"].format(
        USUARIODESTINO_FULLNAME=usuario.fullName,
        LISTA_PREDIOS=codigos_predios,
        USUARIOASIGNA_FULLNAME=asignado_por.split("-")[0])
    enviar_correo(gis, asunto, mensaje, usuario.email, "html")

In [10]:
listado_numeros_prediales = []
listado_numeros_prediales.append("0800101...1")
listado_numeros_prediales.append("0800101...2")
print(str(listado_numeros_prediales).replace("[","").replace("]",""))
if len(listado_numeros_prediales) > 0:
    notificarAsignacion(numero_actualizacion, listado_numeros_prediales, reconocedor, jefe)

'0800101...1', '0800101...2'
usuario: <User username:wmoreno_project_esri_co>
Enviando correo electronico a wmoreno@esri.co...
Obteniendo parámetros de configuracion....
Mensaje enviado satisfactoriamente
